**Text Identification using OpenCV, Tesseract (OCR), and Deep Neural Network.**

In [ ]:
!apt-get install -y tesseract-ocr  # Install Tesseract OCR engine
!pip install pytesseract  # Install pytesseract (Python wrapper for Tesseract)

import cv2          # For image processing
import numpy as np   # Used for numerical operations on image arrays
import pytesseract   # For Optical Character Recognition (OCR)
from google.colab.patches import cv2_imshow  # To display images in Colab
from google.colab import files  # To handle file uploads in Colab
import os           # For file path operations

def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    denoised = cv2.fastNlMeansDenoising(gray)       # Remove noise
    thresh = cv2.threshold(denoised, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]  # Binary thresholding
    return thresh


def detect_text_regions(image):
    # Detecting words
    boxes = pytesseract.image_to_data(image, output_type=pytesseract.Output.DICT)  # Get text region data
    return boxes

def draw_bounding_boxes(image, boxes):
    output = image.copy()  # Copy original image to draw boxes
    n_boxes = len(boxes['level'])  # Total number of detected text regions
    for i in range(n_boxes):
        if int(boxes['conf'][i]) > 60:  # Only consider boxes with confidence > 60%
            (x, y, w, h) = (boxes['left'][i], boxes['top'][i], boxes['width'][i], boxes['height'][i])  # Box coordinates
            cv2.rectangle(output, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Draw rectangle on the image
    return output

def get_detected_text(boxes):
    detected_text = []
    n_boxes = len(boxes['level'])  # Total number of detected boxes
    for i in range(n_boxes):
        if int(boxes['conf'][i]) > 60:  # Only extract text with confidence > 60%
            detected_text.append(boxes['text'][i])  # Append detected text
    return ' '.join(detected_text)  # Combine text into a single string

image_path = "/content/text_img_1.jpg"

image = cv2.imread(image_path)  # Read the image
cv2_imshow(image)

# Check if image was successfully read
if image is None:
    print(f"Error: Unable to read the image file: {image_path}")
else:
    # Display original image
    print("\nOriginal Image:")
    cv2_imshow(image)

    # Preprocess the image and detect text regions
    preprocessed = preprocess_image(image)  # Preprocessing
    boxes = detect_text_regions(preprocessed)  # Detect text regions

    # Draw bounding boxes on original image
    image_with_boxes = draw_bounding_boxes(image, boxes)

    # Display the image with bounding boxes
    print("\nImage with Text Detection Regions:")
    cv2_imshow(image_with_boxes)

    # Extract and print the detected text
    detected_text = get_detected_text(boxes)
    print("\nDetected Text:")
    print(detected_text)

